In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.transform
import pickle
import tensorflow as tf

from flask import Flask, request, render_template

app=Flask(__name__)

UPLOAD_FOLDER = 'upload/'
PICKLE_FOLDER = 'pickle_files/'
MODEL_FOLDER  = 'model/'
    
model= tf.keras.models.load_model(MODEL_FOLDER+"model.h5")
with open (PICKLE_FOLDER+'diego_mapeo.pkl','rb') as f:
    mapping=pickle.load(f)
    
@app.route('/')
def index():
    return render_template('form.html')


@app.route('/upload', methods=['POST'])
def uploader():
    if request.method=='POST':
        
        #getting the image extension
        f=request.files['archivo']
        filename=f.filename
        ext=filename.split('.')[-1]
        
        #checking extension
        if ext in ['jpg','png','jpeg']:
            
            #saving the image
            save_path=f'{UPLOAD_FOLDER}/{filename}'
            f.save(save_path)
            
            #getting the model prediction
            results=model_pipeline(save_path,mapping,model)
            return render_template('form.html',fileupload=True,data=results,image_filename=filename)
        
        return '<H1> Only JPEG, JPG and PNG files <H1>'
    return '<H1> Only POST methods <H1>'

#function to make the prediction
def model_pipeline(file_path,mapping,model):
    img=plt.imread(file_path)
    img=skimage.transform.resize(img,[80,80])
    img=np.array([img])
    predict=model.predict(img)
    return pd.Series(np.round(predict[0],2),index=mapping.values()).sort_values(ascending=False)[:5].to_dict()



if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Oct/2023 13:04:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 13:04:28] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 176ms/step


127.0.0.1 - - [14/Oct/2023 13:04:32] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 13:04:32] "GET /upload/dog_test.jpg HTTP/1.1" 404 -


1/1 [==============================] - 0s 47ms/step


127.0.0.1 - - [14/Oct/2023 13:04:38] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 13:04:38] "GET /upload/lion.jpg HTTP/1.1" 404 -


1/1 [==============================] - 0s 40ms/step


127.0.0.1 - - [14/Oct/2023 13:04:42] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 13:04:42] "GET /upload/eagle.jpg HTTP/1.1" 404 -
